# Text classification task

As mentioned earlier, we will focus on a simple text classification task using the **AG_NEWS** dataset, which involves classifying news headlines into one of four categories: World, Sports, Business, and Sci/Tech.

## The Dataset

This dataset is included in the [`torchtext`](https://github.com/pytorch/text) module, making it easily accessible.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Here, `train_dataset` and `test_dataset` contain collections that return pairs of label (class number) and text respectively, for example:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

So, let's print out the first 10 new headlines from our dataset:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Because datasets are iterators, if we want to use the data multiple times we need to convert it to list:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenization

Now we need to transform text into **numbers** that can be represented as tensors. If we aim for word-level representation, we need to do two things:
* use a **tokenizer** to break the text into **tokens**
* create a **vocabulary** from those tokens.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Using vocabulary, we can easily encode our tokenized string into a set of numbers:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Bag of Words text representation

Since words convey meaning, sometimes we can understand the essence of a text simply by analyzing the individual words, without considering their order in the sentence. For instance, when categorizing news articles, words like *weather* and *snow* are likely to suggest *weather forecast*, whereas words like *stocks* and *dollar* might point to *financial news*.

The **Bag of Words** (BoW) vector representation is the most widely used traditional method for representing text as vectors. Each word is assigned to a specific index in the vector, and the corresponding vector element indicates the number of times that word appears in a given document.

![Image showing how a bag of words vector representation is represented in memory.](../../../../../translated_images/en/bag-of-words-example.606fc1738f1d7ba98a9d693e3bcd706c6e83fa7bf8221e6e90d1a206d82f2ea4.png) 

> **Note**: You can also think of BoW as the sum of all one-hot-encoded vectors for the individual words in the text.

Below is an example of how to create a bag of words representation using the Scikit Learn Python library:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

To compute bag-of-words vector from the vector representation of our AG_NEWS dataset, we can use the following function:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Note:** Here we are using global `vocab_size` variable to specify default size of the vocabulary. Since often vocabulary size is pretty big, we can limit the size of the vocabulary to most frequent words. Try lowering `vocab_size` value and running the code below, and see how it affects the accuracy. You should expect some accuracy drop, but not dramatic, in lieu of higher performance.


## Training BoW classifier

Now that we know how to create a Bag-of-Words representation for our text, let's train a classifier using it. First, we need to prepare our dataset for training by converting all positional vector representations into Bag-of-Words representations. This can be done by using the `bowify` function as the `collate_fn` parameter in the standard torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Now let's define a simple classifier neural network that contains one linear layer. The size of the input vector equals to `vocab_size`, and output size corresponds to the number of classes (4). Because we are solving classification task, the final activation function is `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Now we will define standard PyTorch training loop. Because our dataset is quite large, for our teaching purpose we will train only for one epoch, and sometimes even for less than an epoch (specifying the `epoch_size` parameter allows us to limit training). We would also report accumulated training accuracy during training; the frequency of reporting is specified using `report_freq` parameter.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams and N-Grams

One drawback of the bag of words approach is that some words belong to multi-word expressions. For instance, the term 'hot dog' has a completely different meaning compared to the individual words 'hot' and 'dog' in other contexts. If we always represent the words 'hot' and 'dog' with the same vectors, it can lead to confusion in our model.

To solve this issue, **N-gram representations** are often employed in document classification methods, where the frequency of each word, pair of words, or triplet of words becomes a valuable feature for training classifiers. In a bigram representation, for example, we include all word pairs in the vocabulary in addition to the original words.

Here’s an example of how to create a bigram bag of words representation using Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

The main disadvantage of the N-gram approach is that the size of the vocabulary increases very rapidly. In practice, it is necessary to combine the N-gram representation with dimensionality reduction techniques, such as *embeddings*, which we will cover in the next unit.

To apply the N-gram representation to our **AG News** dataset, we need to create a specific N-gram vocabulary:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


We could then use the same code as above to train the classifier, but it would be very inefficient in terms of memory usage. In the next section, we will train a bigram classifier using embeddings.

> **Note:** You should only keep those ngrams that appear in the text more than a specified number of times. This ensures that rare bigrams are excluded, significantly reducing the dimensionality. To achieve this, set the `min_freq` parameter to a higher value and observe how the vocabulary size changes.


## Term Frequency Inverse Document Frequency TF-IDF

In the BoW representation, all word occurrences are treated equally, regardless of the word itself. However, it's clear that common words like *a*, *in*, etc., are far less significant for classification compared to specialized terms. In fact, in most NLP tasks, certain words carry more importance than others.

**TF-IDF** stands for **term frequency–inverse document frequency**. It is a variation of the bag-of-words approach, where instead of using a binary 0/1 value to indicate whether a word appears in a document, a floating-point value is used that reflects the frequency of the word in the corpus.

More formally, the weight $w_{ij}$ of a word $i$ in document $j$ is defined as:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
where:
* $tf_{ij}$ is the number of times word $i$ appears in document $j$, which corresponds to the BoW value we discussed earlier.
* $N$ is the total number of documents in the collection.
* $df_i$ is the number of documents in the collection that contain word $i$.

The TF-IDF value $w_{ij}$ increases proportionally to the frequency of the word in a document but is offset by the number of documents in the corpus that contain the word. This adjustment accounts for the fact that some words are more common than others. For instance, if a word appears in *every* document in the collection, $df_i=N$, and $w_{ij}=0$, meaning such terms would be completely ignored.

You can easily generate TF-IDF vectorization for text using Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Conclusion

Although TF-IDF representations assign frequency weights to different words, they cannot capture meaning or order. As the renowned linguist J. R. Firth stated in 1935, “The complete meaning of a word is always contextual, and no study of meaning apart from context can be taken seriously.” Later in the course, we will explore how to extract contextual information from text using language modeling.



---

**Disclaimer**:  
This document has been translated using the AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator). While we aim for accuracy, please note that automated translations may include errors or inaccuracies. The original document in its native language should be regarded as the authoritative source. For critical information, professional human translation is advised. We are not responsible for any misunderstandings or misinterpretations resulting from the use of this translation.
